In [1]:

from simen_funksjoner import *
from read_csv import *

In [2]:
h2o.init(ip ='localhost', port = 54321)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.10" 2021-01-19; OpenJDK Runtime Environment AdoptOpenJDK (build 11.0.10+9); OpenJDK 64-Bit Server VM AdoptOpenJDK (build 11.0.10+9, mixed mode)
  Starting server from /Users/simenvoldqvam/opt/anaconda3/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/z4/blr9pbks7g1gwx7yzstbjlgw0000gn/T/tmp3xuvjkx2
  JVM stdout: /var/folders/z4/blr9pbks7g1gwx7yzstbjlgw0000gn/T/tmp3xuvjkx2/h2o_simenvoldqvam_started_from_python.out
  JVM stderr: /var/folders/z4/blr9pbks7g1gwx7yzstbjlgw0000gn/T/tmp3xuvjkx2/h2o_simenvoldqvam_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,13 days
H2O_cluster_name:,H2O_from_python_simenvoldqvam_spmoau
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [10]:
point = make_point(stores_train)
test_point = make_point(stores_test)

stores_gk_impuded_merged = stores_gk_impuded.merge(point,how='left', on='store_id')
stores_gk_impuded_merged_df = stores_gk_impuded_merged.drop(['lat','lon'], axis = 1)
stores_gk_impuded_merged_df = stores_gk_impuded_merged_df[stores_gk_impuded_merged_df.revenue != 0]




In [4]:


stores_train_hf = h2o.H2OFrame(stores_gk_impuded_merged_df)
stores_test_hf = h2o.H2OFrame(stores_test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:
#very simple training model with h2o
x = stores_train_hf.columns
y = 'revenue'
x.remove(y)

aml = H2OAutoML(max_models=3, seed=1)
aml.train(x=x, y=y, training_frame= stores_train_hf) 

AutoML progress: |
16:04:59.10: _train param, Dropping bad and constant columns: [store_id, year, store_name]

██
16:05:15.157: _train param, Dropping bad and constant columns: [store_id, year, store_name]

███
16:05:37.747: GLM_1_AutoML_1_20221109_160458 [GLM def_1] failed: DistributedException from /127.0.0.1:54321: 'Java heap space', caused by java.lang.OutOfMemoryError: Java heap space
16:05:37.759: _train param, Dropping bad and constant columns: [store_id, year, store_name]

█
16:05:46.105: _train param, Dropping bad and constant columns: [store_id, year, store_name]

█
16:05:56.290: _train param, Dropping unused columns: [store_id, year, store_name]
16:05:57.310: _train param, Dropping unused columns: [store_id, year, store_name]

████████████████████████████████████████████████████████| (done) 100%


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,4.8087564,0.1768588,5.069401,4.747088,4.798762,4.8473444,4.5811853
mean_residual_deviance,95.33758,13.267868,110.21431,105.10906,92.525055,92.81728,76.022194
mse,95.33758,13.267868,110.21431,105.10906,92.525055,92.81728,76.022194
null_deviance,608207.75,51423.188,631711.7,639282.1,565429.7,662139.6,542475.6
r2,0.5964419,0.0485666,0.5421476,0.5743245,0.5710865,0.6379275,0.6567234
residual_deviance,245814.19,39243.95,288981.9,271917.16,242508.17,239561.4,186102.33
rmse,9.744562,0.6901948,10.498301,10.252271,9.618995,9.634172,8.71907
rmsle,0.8098572,0.0109270,0.8194412,0.8218924,0.8060617,0.7951293,0.8067616


In [6]:
lb = h2o.automl.get_leaderboard(aml, extra_columns = "ALL")
best_model = aml.get_best_model()

lb

model_id,rmse,mse,mae,rmsle,mean_residual_deviance,algo
StackedEnsemble_AllModels_1_AutoML_1_20221109_160458,9.78854,95.8156,4.79642,0.805706,95.8156,StackedEnsemble
StackedEnsemble_BestOfFamily_1_AutoML_1_20221109_160458,9.81619,96.3577,4.82863,nan,96.3577,StackedEnsemble
GBM_1_AutoML_1_20221109_160458,10.1154,102.322,4.91817,nan,102.322,GBM
XGBoost_1_AutoML_1_20221109_160458,10.17,103.429,5.10407,nan,103.429,XGBoost
XGBoost_2_AutoML_1_20221109_160458,10.1932,103.901,5.02674,nan,103.901,XGBoost


In [7]:
m = h2o.get_model(lb[2,"model_id"])
print(m.varimp(use_pandas=True))


               variable  relative_importance  scaled_importance  percentage
0            chain_name         7.189118e+06           1.000000    0.573460
1     municipality_name         2.144072e+06           0.298239    0.171028
2   plaace_hierarchy_id         1.470943e+06           0.204607    0.117334
3    sales_channel_name         7.286248e+05           0.101351    0.058121
4             mall_name         3.737198e+05           0.051984    0.029811
5         district_name         2.712054e+05           0.037724    0.021633
6               address         1.303064e+05           0.018125    0.010394
7                 point         1.004960e+05           0.013979    0.008016
8              area_km2         6.853793e+04           0.009534    0.005467
9       grunnkrets_name         5.072388e+04           0.007056    0.004046
10        grunnkrets_id         8.649481e+03           0.001203    0.000690
